In [1]:
print("Good Morning!")

Good Morning!


Descriptors exist to give developers fine-grained control over what happens when an attribute is **accessed, set, or deleted**. They are particularly useful for scenarios where the behavior of an attribute needs to be customized, automated, or abstracted away from the core logic of a class.

Here's a breakdown of why descriptors are necessary and when they shine:

---

### **1. Reusable Logic Across Attributes**
Instead of repeating getter, setter, and deleter logic in multiple places, descriptors centralize this logic into a single class. For instance, if you need to validate or transform data for several attributes, descriptors let you reuse the same logic.

#### Example:
```python
class PositiveValue:
    def __get__(self, instance, owner):
        return instance.__dict__.get(self.name, 0)

    def __set__(self, instance, value):
        if value < 0:
            raise ValueError("Value must be positive")
        instance.__dict__[self.name] = value

    def __set_name__(self, owner, name):
        self.name = name  # Dynamically bind to attribute name


class Account:
    balance = PositiveValue()  # Reusable descriptor


a = Account()
a.balance = 100  # Works fine
print(a.balance)  # Prints 100
a.balance = -50  # Raises ValueError
```

---

### **2. Encapsulation of Complex Behaviors**
Descriptors allow you to hide complex logic behind simple attribute access. For example, if an attribute's value depends on external data (like a database or API), descriptors abstract that complexity.

#### Example:
```python
class DatabaseField:
    def __get__(self, instance, owner):
        return f"Fetching value from database for {self.name}"

    def __set_name__(self, owner, name):
        self.name = name


class User:
    username = DatabaseField()  # Descriptor simulating a DB field
    email = DatabaseField()


user = User()
print(user.username)  # Simulates fetching the 'username' field
```

---

### **3. Integration with Built-in Python Features**
Descriptors are the foundation of many powerful built-in Python features:
- `@property`: Wraps `__get__`, `__set__`, and `__delete__`.
- `@staticmethod` and `@classmethod`: Control access to static and class-level methods.
- Functions themselves are descriptors, enabling bound methods.

---

### **4. Lazy Initialization and Caching**
Descriptors make lazy initialization and caching possible by controlling when and how values are calculated or stored.

#### Example:
```python
class LazyAttribute:
    def __get__(self, instance, owner):
        value = instance.compute_expensive_value()  # Lazy computation
        instance.__dict__[self.name] = value  # Cache the result
        return value

    def __set_name__(self, owner, name):
        self.name = name


class MyClass:
    expensive_attr = LazyAttribute()

    def compute_expensive_value(self):
        print("Computing value...")
        return 42


obj = MyClass()
print(obj.expensive_attr)  # Triggers computation
print(obj.expensive_attr)  # Uses cached value
```

---

### **5. Fine-grained Control for Advanced Scenarios**
Descriptors are critical for building frameworks and tools that need fine-grained control over attribute access, such as:
- ORM libraries like SQLAlchemy or Django ORM.
- Frameworks for validations, logging, or dependency injection.
- Proxy objects or dynamic attribute delegation.

---

### **Why Not Just Use `@property`?**
Descriptors provide greater flexibility than `@property`:
- They can be reused across classes.
- They can control behavior at the class level, not just instance attributes.
- They work seamlessly with metaprogramming and dynamic class creation.

### Summary
Descriptors exist because Python needed a flexible, reusable, and powerful mechanism to handle attributes dynamically. They enable developers to abstract behavior, enforce constraints, and manage state efficiently. If your use case doesn't involve such needs, you may rarely need to create custom descriptors yourself, but you'll often rely on frameworks and libraries that use them under the hood.

In [2]:
from datetime import datetime

class TimeUTC:
    def __get__(self, instance, owner_class):
        return datetime.utcnow().isoformat()

In [8]:
class Logger:
    current_time = TimeUTC()

In [9]:
Logger.__dict__

mappingproxy({'__module__': '__main__',
              'current_time': <__main__.TimeUTC at 0x20727bd0e50>,
              '__dict__': <attribute '__dict__' of 'Logger' objects>,
              '__weakref__': <attribute '__weakref__' of 'Logger' objects>,
              '__doc__': None})

In [10]:
l = Logger()

In [12]:
l.current_time

'2024-12-11T01:17:10.106716'

In [13]:
Logger.current_time

'2024-12-11T01:17:20.135256'

In [14]:
from random import choice, seed

class Deck:
    @property
    def suit(self):
        return choice(('Spade', 'Heart', 'Diamond', 'Club'))
        
    @property
    def card(self):
        return choice(tuple('23456789JQKA') + ('10',))

In [15]:
d = Deck()

In [16]:
seed(0)

for _ in range(10):
    print(d.card, d.suit)

8 Club
2 Diamond
J Club
8 Diamond
9 Diamond
Q Heart
J Heart
6 Heart
10 Spade
Q Diamond


In [17]:
class Choice:
    def __init__(self, *choices):
        self.choices = choices
        
    def __get__(self, instance, owner_class):
        return choice(self.choices)

In [18]:
class Deck:
    suit = Choice('Spade', 'Heart', 'Diamond', 'Club')
    card = Choice(*'23456789JQKA', '10')

In [19]:
seed(0)

d = Deck()

for _ in range(10):
    print(d.card, d.suit)

8 Club
2 Diamond
J Club
8 Diamond
9 Diamond
Q Heart
J Heart
6 Heart
10 Spade
Q Diamond


In [20]:
class Dice:
    die_1 = Choice(1,2,3,4,5,6)
    die_2 = Choice(1,2,3,4,5,6)
    die_3 = Choice(1,2,3,4,5,6)

In [21]:
seed(0)

dice = Dice()
for _ in range(10):
    print(dice.die_1, dice.die_2, dice.die_3)

4 4 1
3 5 4
4 3 4
3 5 2
5 2 3
2 1 5
3 5 6
5 2 3
1 6 1
6 3 4


In [22]:
from datetime import datetime

class TimeUTC:
    def __get__(self, instance, owner_class):
        print(f'__get__ called, self={self}, instance={instance}, owner_class={owner_class}')
        return datetime.utcnow().isoformat()

In [23]:
class Logger1:
    current_time = TimeUTC()
    
class Logger2:
    current_time = TimeUTC()

In [24]:
Logger1.current_time

__get__ called, self=<__main__.TimeUTC object at 0x0000020727BD1100>, instance=None, owner_class=<class '__main__.Logger1'>


'2024-12-11T01:25:06.643590'

In [25]:
Logger2.current_time

__get__ called, self=<__main__.TimeUTC object at 0x0000020727BD17F0>, instance=None, owner_class=<class '__main__.Logger2'>


'2024-12-11T01:25:41.470670'

In [26]:
l1 = Logger1()
print(hex(id(l1)))

0x20727bd1c70


In [27]:
l1.current_time

__get__ called, self=<__main__.TimeUTC object at 0x0000020727BD1100>, instance=<__main__.Logger1 object at 0x0000020727BD1C70>, owner_class=<class '__main__.Logger1'>


'2024-12-11T01:26:03.593742'

In [28]:
l2 = Logger2()
print(hex(id(l2)))
l2.current_time

0x20727bd0eb0
__get__ called, self=<__main__.TimeUTC object at 0x0000020727BD17F0>, instance=<__main__.Logger2 object at 0x0000020727BD0EB0>, owner_class=<class '__main__.Logger2'>


'2024-12-11T01:26:26.387831'

In [29]:
from datetime import datetime

class TimeUTC:
    def __get__(self, instance, owner_class):
        if instance is None:
            # called from class
            return self
        else:
            # called from instance
            return datetime.utcnow().isoformat()

In [30]:
class Logger:
    current_time = TimeUTC()

In [31]:
Logger.current_time

In [32]:
l = Logger()

In [33]:
l.current_time

'2024-12-11T01:27:25.383246'

In [34]:
class Logger:
    @property
    def current_time(self):
        return datetime.utcnow().isoformat()

In [35]:
Logger.current_time

In [36]:
l = Logger()
l.current_time

'2024-12-11T01:28:19.445070'

In [37]:
class TimeUTC:
    def __get__(self, instance, owner_class):
        if instance is None:
            # called from class
            return self
        else:
            # called from instance
            print(f'__get__ called in {self}')
            return datetime.utcnow().isoformat()
        
class Logger:
    current_time = TimeUTC()

In [38]:
l1 = Logger()
l2 = Logger()

In [39]:
l1.current_time, l2.current_time

__get__ called in <__main__.TimeUTC object at 0x0000020727AF2670>
__get__ called in <__main__.TimeUTC object at 0x0000020727AF2670>


('2024-12-11T01:28:52.511548', '2024-12-11T01:28:52.511548')

In [40]:
class Countdown:
    def __init__(self, start):
        self.start = start + 1
        
    def __get__(self, instance, owner):
        if instance is None:
            return self
        else:
            self.start -= 1
            return self.start

In [41]:
class Rocket:
    countdown = Countdown(10)

In [42]:
rocket1 = Rocket()
rocket2 = Rocket()

In [43]:
rocket1.countdown

10

In [44]:
rocket2.countdown

9

In [45]:
rocket1.countdown

8

In [46]:
class IntegerValue:
    def __set__(self, instance, value):
        print(f'__set__ called, instance={instance}, value={value}')
        
    def __get__(self, instance, owner_class):
        if instance is None:
            print('__get__ called from class')
        else:
            print(f'__get__ called, instance={instance}, owner_class={owner_class}')

In [47]:
class Point2D:
    x = IntegerValue()
    y = IntegerValue()

In [48]:
Point2D.x

__get__ called from class


In [49]:
p = Point2D()

In [50]:
p.x

__get__ called, instance=<__main__.Point2D object at 0x0000020727BD16D0>, owner_class=<class '__main__.Point2D'>


In [51]:
p.x = 100

__set__ called, instance=<__main__.Point2D object at 0x0000020727BD16D0>, value=100


In [52]:
class IntegerValue:
    def __set__(self, instance, value):
        self._value = int(value)
        
    def __get__(self, instance, owner_class):
        if instance is None:
            return self
        else:
            return self._value

In [53]:
class Point2D:
    x = IntegerValue()
    y = IntegerValue()

In [54]:
p1 = Point2D()

In [55]:
p1.x = 1.1
p1.y = 2.2

In [56]:
p1.x, p1.y

(1, 2)

In [57]:
p2 = Point2D()

In [58]:
p2.x, p2.y

(1, 2)

In [59]:
p2.x = 100.9

In [60]:
p2.x, p1.x

(100, 100)

In [61]:
class IntegerValue:
    def __set__(self, instance, value):
        instance.stored_value = int(value)
        
    def __get__(self, instance, owner_class):
        if instance is None:
            return self
        else:
            return getattr(instance, 'stored_value', None)

In [62]:
class Point1D:
    x = IntegerValue()

In [63]:
p1, p2 = Point1D(), Point1D()

In [64]:
p1.x = 10.1
p2.x = 20.2

In [65]:
p1.x, p2.x

(10, 20)

In [66]:
p1.__dict__, p2.__dict__

({'stored_value': 10}, {'stored_value': 20})

In [67]:
class Point2D:
    x = IntegerValue()
    y = IntegerValue()

In [68]:
p = Point2D()

In [75]:
Point2D.__dict__

mappingproxy({'__module__': '__main__',
              'x': <__main__.IntegerValue at 0x20727ab3730>,
              'y': <__main__.IntegerValue at 0x20727bea9a0>,
              '__dict__': <attribute '__dict__' of 'Point2D' objects>,
              '__weakref__': <attribute '__weakref__' of 'Point2D' objects>,
              '__doc__': None})

In [69]:
p.x = 10.1

In [70]:
p.__dict__

{'stored_value': 10}

In [71]:
p.y = 20.2

In [72]:
p.__dict__

{'stored_value': 20}

In [73]:
p.x, p.y

(20, 20)

In [74]:
class IntegerValue:
    def __init__(self, name):
        self.storage_name = '_' + name 
        
    def __set__(self, instance, value):
        setattr(instance, self.storage_name, int(value))
        
    def __get__(self, instance, owner_class):
        if instance is None:
            return self
        else:
            return getattr(instance, self._storage_name, None)
        
class Point2D:
    x = IntegerValue('x')
    y = IntegerValue('y')

In [76]:
p1 = Point2D()
p2 = Point2D()

In [77]:
p1.x = 10.1
p1.y = 20.2

In [78]:
p1.__dict__

{'_x': 10, '_y': 20}

In [79]:
p2.x = 100.1
p2.y = 200.2

In [80]:
p2.__dict__

{'_x': 100, '_y': 200}

In [81]:
class IntegerValue:
    def __init__(self):
        self.values = {}
        
    def __set__(self, instance, value):
        self.values[instance] = int(value)
        
    def __get__(self, instance, owner_class):
        if instance is None:
            return self
        else:
            return self.values.get(instance)

In [82]:
class Point2D:
    x = IntegerValue()
    y = IntegerValue()

In [83]:
p1 = Point2D()
p2 = Point2D()

In [84]:
p1.x = 10.1
p1.y = 20.2

In [85]:
p1.x, p1.y

(10, 20)

In [86]:
Point2D.x.values

{<__main__.Point2D at 0x20727a91df0>: 10}

In [88]:
Point2D.y.values

{<__main__.Point2D at 0x20727a91df0>: 20}

In [89]:
hex(id(p1))

'0x20727a91df0'

In [90]:
p2 = Point2D()
p2.x = 100.1
p2.y = 200.2

In [91]:
hex(id(p2))

'0x20727bea130'

In [92]:
Point2D.x.values

{<__main__.Point2D at 0x20727a91df0>: 10,
 <__main__.Point2D at 0x20727bea130>: 100}

In [93]:
Point2D.y.values

{<__main__.Point2D at 0x20727a91df0>: 20,
 <__main__.Point2D at 0x20727bea130>: 200}

In [94]:
p1.x, p1.y, p2.x, p2.y

(10, 20, 100, 200)

# Strong and Weak References

In [95]:
import ctypes

def ref_count(address):
    return ctypes.c_long.from_address(address).value

In [96]:
p1 = Point2D()
id_p1 = id(p1)

In [97]:
ref_count(id_p1)

1

In [98]:
p1.x = 100.1

In [99]:
ref_count(id_p1)

2

In [100]:
'p1' in globals()

True

In [101]:
del p1

In [102]:
'p1' in globals()

False

In [103]:
ref_count(id_p1)

1

In [104]:
Point2D.x.values.items()

dict_items([(<__main__.Point2D object at 0x0000020727A91DF0>, 10), (<__main__.Point2D object at 0x0000020727BEA130>, 100), (<__main__.Point2D object at 0x0000020727BEA5B0>, 100)])

In [105]:
hex(id_p1)

'0x20727bea5b0'

In [106]:
import ctypes

def ref_count(address):
    return ctypes.c_long.from_address(address).value

In [107]:
class Person:
    def __init__(self, name):
        self.name = name
        
    def __repr__(self):
        return f'Person(name={self.name})'

In [108]:
p1 = Person('Guido')
p2 = p1

In [109]:
p1_id = id(p1)
p2_id = id(p2)

In [110]:
p1_id == p2_id, ref_count(p1_id)

(True, 2)

In [111]:
del p2

In [112]:
ref_count(p1_id)

1

In [113]:
del p1

In [114]:
ref_count(p1_id)

-1899931512

In [115]:
import weakref

In [116]:
p1 = Person('Guido')

In [117]:
p1_id = id(p1)

In [118]:
ref_count(p1_id)

1

In [119]:
p2 = p1

In [120]:
ref_count(p1_id)

2

In [121]:
weak1 = weakref.ref(p1)

In [122]:
ref_count(p1_id)

2

In [123]:
weak1

<weakref at 0x0000020728CCD0E0; to 'Person' at 0x0000020727BEA8E0>

In [124]:
hex(p1_id)

'0x20727bea8e0'

In [125]:
weak1 is p1

False

In [126]:
ref_count(p1_id)

2

In [127]:
print(weak1())

Person(name=Guido)


In [128]:
ref_count(p1_id)

2

In [129]:
p3 = weak1()

In [130]:
p1 is p3

True

In [131]:
ref_count(p1_id)


3

In [132]:
weakref.getweakrefcount(p1), ref_count(p1_id)

(1, 3)

In [133]:
import sys

In [134]:
sys.getrefcount(p1)

4

In [135]:
del p3
del p2

In [136]:
ref_count(p1_id)

1

In [137]:
del p1

In [138]:
weak1

<weakref at 0x0000020728CCD0E0; dead>

In [139]:
obj = weak1()

In [140]:
obj is None

True

In [141]:
l = [1, 2, 3]
try:
    w = weakref.ref(l)
except TypeError as ex:
    print(ex)

cannot create weak reference to 'list' object


In [142]:
l = {'a': 1}
try:
    w = weakref.ref(l)
except TypeError as ex:
    print(ex)

cannot create weak reference to 'dict' object


In [143]:
l = 100
try:
    w = weakref.ref(l)
except TypeError as ex:
    print(ex)


cannot create weak reference to 'int' object


In [144]:
l = 'python'
try:
    w = weakref.ref(l)
except TypeError as ex:
    print(ex)

cannot create weak reference to 'str' object


In [145]:
p1 = Person('Guido')

In [146]:
d = weakref.WeakKeyDictionary()

In [147]:
ref_count(id(p1))

1

In [148]:
weakref.getweakrefcount(p1)

0

In [149]:
d[p1] = 'Guido'

In [150]:
ref_count(id(p1)), weakref.getweakrefcount(p1)

(1, 1)

In [151]:
hex(id(p1)), list(d.keyrefs())

('0x20727be3310',
 [<weakref at 0x0000020728DA5EA0; to 'Person' at 0x0000020727BE3310>])

In [152]:
del p1

In [153]:
list(d.keyrefs())

[]

In [154]:
try:
    d['python'] = 'test'
except TypeError as ex:
    print(ex)

cannot create weak reference to 'str' object


In [155]:
class Person:
    def __init__(self, name):
        self.name = name
        
    def __eq__(self, other):
        return isinstance(other, Person) and self.name == other.name

In [156]:
p1 = Person('Guido')
p2 = Person('Guido')

In [157]:
p1 == p2

True

In [158]:
try:
    hash(p1)
except TypeError as ex:
    print(ex)

unhashable type: 'Person'


In [159]:
try:
    d[p1] = 'Guido'
except TypeError as ex:
    print(ex)

unhashable type: 'Person'


In [160]:
class IntegerValue:
    def __init__(self):
        self.values = {}
        
    def __set__(self, instance, value):
        self.values[instance] = int(value)
        
    def __get__(self, instance, owner_class):
        if instance is None:
            return self
        else:
            return self.values.get(instance)

In [161]:
import weakref

In [162]:
class IntegerValue:
    def __init__(self):
        self.values = weakref.WeakKeyDictionary()
        
    def __set__(self, instance, value):
        self.values[instance] = int(value)
        
    def __get__(self, instance, owner_class):
        if instance is None:
            return self
        else:
            return self.values.get(instance)

In [163]:
class Point:
    x = IntegerValue()

In [164]:
p = Point()
print(hex(id(p)))

0x20727be3a90


In [165]:
p.x = 100.1

In [166]:
p.x

100

In [167]:
Point.x.values.keyrefs()

[<weakref at 0x0000020728DC3810; to 'Point' at 0x0000020727BE3A90>]

In [168]:
del p

In [169]:
Point.x.values.keyrefs()

[]

In [205]:
class IntegerValue:
    def __init__(self):
        self.values = {}
        
    def __set__(self, instance, value):
        self.values[id(instance)] = int(value)
        
    def __get__(self, instance, owner_class):
        if instance is None:
            return self
        else:
            return self.values.get(id(instance))

In [206]:
class Point:
    x = IntegerValue()
    
    def __init__(self, x):
        self.x = x
        
    def __eq__(self, other):
        return isinstance(other, Point) and self.x == other.x

In [207]:
p = Point(10.1)

In [208]:
p.x

10

In [209]:
p.x = 20.2

In [210]:
p.x

20

In [211]:
id(p), Point.x.values

(2229754724896, {2229754724896: 20})

In [212]:
import ctypes

def ref_count(address):
    return ctypes.c_long.from_address(address).value

In [213]:
p_id = id(p)

In [214]:
ref_count(p_id)

1

In [215]:
del p

In [216]:
ref_count(p_id)

2

In [217]:
Point.x.values

{2229754724896: 20}

In [218]:
p = Point(10.1)
weak_p = weakref.ref(p)

In [219]:
print(hex(id(p)), weak_p)  
# again note how I need to use print to avoid affecting the ref count

0x20727be33d0 <weakref at 0x0000020728D71900; to 'Point' at 0x0000020727BE33D0>


In [220]:
ref_count(id(p))

1

In [221]:
del p

In [222]:
print(weak_p)

<weakref at 0x0000020728D71900; dead>


In [223]:
def obj_destroyed(obj):
    print(f'{obj} is being destroyed')

In [224]:
p = Point(10.1)
w = weakref.ref(p, obj_destroyed)

In [225]:
del p

<weakref at 0x0000020728D8A3B0; dead> is being destroyed


In [226]:
class IntegerValue:
    def __init__(self):
        self.values = {}
        
    def __set__(self, instance, value):
        self.values[id(instance)] = (weakref.ref(instance, self._remove_object), 
                                     int(value)
                                    )
        
    def __get__(self, instance, owner_class):
        if instance is None:
            return self
        else:
            value_tuple = self.values.get(id(instance))
            return value_tuple[1]  # return the associated value, not the weak ref
        
    def _remove_object(self, weak_ref):
        print(f'removing dead entry for {weak_ref}')
        # how do we find that weak reference?

In [227]:
class Point:
    x = IntegerValue()

In [228]:
p1 = Point()
p2 = Point()

In [229]:
p1.x, p2.x = 10.1, 100.1

In [230]:
p1.x, p2.x

(10, 100)

In [231]:
ref_count(id(p1)), ref_count(id(p2))

(1, 1)

In [232]:
del p1

removing dead entry for <weakref at 0x0000020728DADEA0; dead>


In [233]:
del p2

removing dead entry for <weakref at 0x0000020728D82770; dead>


In [234]:
class IntegerValue:
    def __init__(self):
        self.values = {}
        
    def __set__(self, instance, value):
        self.values[id(instance)] = (weakref.ref(instance, self._remove_object), 
                                     int(value)
                                    )
        
    def __get__(self, instance, owner_class):
        if instance is None:
            return self
        else:
            value_tuple = self.values.get(id(instance))
            return value_tuple[1]  # return the associated value, not the weak ref
        
    def _remove_object(self, weak_ref):
        reverse_lookup = [key for key, value in self.values.items()
                         if value[0] is weak_ref]
        if reverse_lookup:
            # key found
            key = reverse_lookup[0]
            del self.values[key]

In [235]:
class Point:
    x = IntegerValue()

In [236]:
p = Point()

In [237]:
p.x = 10.1

In [238]:
p.x

10

In [239]:
Point.x.values

{2229754805120: (<weakref at 0x0000020728DC3310; to 'Point' at 0x0000020727BE3B80>,
  10)}

In [240]:
ref_count(id(p))

1

In [241]:
del p

In [242]:
Point.x.values

{}

In [243]:
class Person:
    pass

In [244]:
Person.__dict__

mappingproxy({'__module__': '__main__',
              '__dict__': <attribute '__dict__' of 'Person' objects>,
              '__weakref__': <attribute '__weakref__' of 'Person' objects>,
              '__doc__': None})

In [245]:
hasattr(Person.__weakref__, '__get__'), hasattr(Person.__weakref__, '__set__')

(True, True)

In [246]:
p = Person()

In [247]:
hasattr(p, '__weakref__')

True

In [248]:
print(p.__weakref__)

None


In [249]:
w = weakref.ref(p)

In [250]:
p.__weakref__

<weakref at 0x0000020728DCD360; to 'Person' at 0x0000020727A91B80>

In [251]:
class Person:
    __slots__ = 'name',

In [252]:
Person.__dict__

mappingproxy({'__module__': '__main__',
              '__slots__': ('name',),
              'name': <member 'name' of 'Person' objects>,
              '__doc__': None})

In [253]:
p = Person()

In [254]:
hasattr(p, '__weakref__')

False

In [255]:
try:
    weakref.ref(p)
except TypeError as ex:
    print(ex)

cannot create weak reference to 'Person' object


In [256]:
class Person:
    __slots__ = 'name', '__weakref__'

In [257]:
Person.__dict__

mappingproxy({'__module__': '__main__',
              '__slots__': ('name', '__weakref__'),
              'name': <member 'name' of 'Person' objects>,
              '__weakref__': <attribute '__weakref__' of 'Person' objects>,
              '__doc__': None})

In [258]:
p = Person()

In [259]:
hasattr(p, '__weakref__')

True

In [260]:
w = weakref.ref(p)

In [261]:
class ValidString:
    def __init__(self, min_length=0, max_length=255):
        self.data = {}
        self._min_length = min_length
        self._max_length = max_length
        
    def __set__(self, instance, value):
        if not isinstance(value, str):
            raise ValueError('Value must be a string.')
        if len(value) < self._min_length:
            raise ValueError(
                f'Value should be at least {self._min_length} characters.'
            )
        if len(value) > self._max_length:
            raise ValueError(
                f'Value cannot exceed {self._max_length} characters.'
            )
        self.data[id(instance)] = (weakref.ref(instance, self._finalize_instance), 
                                   value
                                  )
        
    def __get__(self, instance, owner_class):
        if instance is None:
            return self
        else:
            value_tuple = self.data.get(id(instance))
            return value_tuple[1]  
        
    def _finalize_instance(self, weak_ref):
        reverse_lookup = [key for key, value in self.data.items()
                         if value[0] is weak_ref]
        if reverse_lookup:
            # key found
            key = reverse_lookup[0]
            del self.data[key]

In [262]:
class Person:
    __slots__ = '__weakref__',
    
    first_name = ValidString(1, 100)
    last_name = ValidString(1, 100)
    
    def __eq__(self, other):
        return (
            isinstance(other, Person) and 
            self.first_name == other.first_name and 
            self.last_name == other.last_name
        )
    
class BankAccount:
    __slots__ = '__weakref__',
    
    account_number = ValidString(5, 255)
    
    def __eq__(self, other):
        return (
            isinstance(other, BankAccount) and 
            self.account_number == other.account_number
        )

In [263]:
p1 = Person()

In [264]:
try:
    p1.first_name = ''
except ValueError as ex:
    print(ex)

Value should be at least 1 characters.


In [265]:
p2 = Person()

In [266]:
p1.first_name, p1.last_name = 'Guido', 'van Rossum'
p2.first_name, p2.last_name = 'Raymond', 'Hettinger'

In [267]:
b1, b2 = BankAccount(), BankAccount()

In [268]:
b1.account_number, b2.account_number = 'Savings', 'Checking'

In [269]:
p1.first_name, p1.last_name

('Guido', 'van Rossum')

In [270]:
p2.first_name, p2.last_name

('Raymond', 'Hettinger')

In [271]:
b1.account_number, b2.account_number

('Savings', 'Checking')

In [272]:
Person.first_name.data

{2229753588272: (<weakref at 0x0000020728DC3900; to 'Person' at 0x0000020727ABAA30>,
  'Guido'),
 2229753588032: (<weakref at 0x0000020728DD9C70; to 'Person' at 0x0000020727ABA940>,
  'Raymond')}

In [273]:
Person.last_name.data

{2229753588272: (<weakref at 0x0000020728DD9DB0; to 'Person' at 0x0000020727ABAA30>,
  'van Rossum'),
 2229753588032: (<weakref at 0x0000020728DD9D60; to 'Person' at 0x0000020727ABA940>,
  'Hettinger')}

In [274]:
del p1
del p2
del b1
del b2

In [275]:
Person.first_name.data

{}

# `__set_name__` Method

In [276]:
class ValidString:
    def __set_name__(self, owner_class, property_name):
        print(f'__set_name__ called: owner={owner_class}, prop={property_name}')

In [277]:
class Person:
    name = ValidString()

__set_name__ called: owner=<class '__main__.Person'>, prop=name


In [278]:
class ValidString:
    def __set_name__(self, owner_class, property_name):
        print(f'__set_name__ called: owner={owner_class}, prop={property_name}')
        self.property_name = property_name
        
    def __get__(self, instance, owner_class):
        if instance is None:
            return self
        else:
            print(f'__get__ called for property {self.property_name} '
                  f'of instance {instance}')

In [279]:
class Person:
    first_name = ValidString()
    last_name = ValidString()

__set_name__ called: owner=<class '__main__.Person'>, prop=first_name
__set_name__ called: owner=<class '__main__.Person'>, prop=last_name


In [280]:
p = Person()

In [281]:
p.first_name

__get__ called for property first_name of instance <__main__.Person object at 0x0000020727BB3D90>


In [282]:
p.last_name

__get__ called for property last_name of instance <__main__.Person object at 0x0000020727BB3D90>


In [283]:
class ValidString():
    def __init__(self, min_length):
        self.min_length = min_length
        
    def __set_name__(self, owner_class, property_name):
        self.property_name = property_name

    def __set__(self, instance, value):
        if not isinstance(value, str):
            raise ValueError(f'{self.property_name} must be a string.')
        if len(value) < self.min_length:
            raise ValueError(f'{self.property_name} must be at least '
                             f'{self.min_length} characters'
                            )
        key = '_' + self.property_name
        setattr(instance, key, value)
        
    def __get__(self, instance, owner_class):
        if instance is None:
            return self
        else:
            key = '_' + self.property_name
            return getattr(instance, key, None)

In [284]:
class Person:
    first_name = ValidString(1)
    last_name = ValidString(2)

In [285]:
p = Person()

In [286]:
try:
    p.first_name = 'Alex'
    p.last_name = 'M'
except ValueError as ex:
    print(ex)

last_name must be at least 2 characters


In [287]:
p = Person()
p.first_name = 'Alex'

In [288]:
p.first_name, p.__dict__

('Alex', {'_first_name': 'Alex'})

In [289]:
p = Person()

In [290]:
p._first_name = 'some data I need to store'

In [291]:
p.__dict__

{'_first_name': 'some data I need to store'}

In [292]:
p.first_name = 'Alex'

In [293]:
p.__dict__

{'_first_name': 'Alex'}

In [294]:
class BankAccount:
    apr = 10

In [295]:
b = BankAccount()

In [296]:
b.apr, b.__dict__

(10, {})

In [297]:
b.apr = 20

In [298]:
b.apr, b.__dict__

(20, {'apr': 20})

In [299]:
class ValidString:
    def __init__(self, min_length):
        self.min_length = min_length
        
    def __set_name__(self, owner_class, property_name):
        self.property_name = property_name

    def __set__(self, instance, value):
        if not isinstance(value, str):
            raise ValueError(f'{self.property_name} must be a string.')
        if len(value) < self.min_length:
            raise ValueError(f'{self.property_name} must be at least '
                             f'{self.min_length} characters'
                            )
        instance.__dict__[self.property_name] = value
        
    def __get__(self, instance, owner_class):
        if instance is None:
            return self
        else:
            print (f'calling __get__ for {self.property_name}')
            return instance.__dict__.get(self.property_name, None)

In [300]:
class Person:
    first_name = ValidString(1)
    last_name = ValidString(2)

In [301]:
p = Person()

In [302]:
p.__dict__

{}

In [303]:
p.first_name = 'Alex'

In [304]:
p.__dict__

{'first_name': 'Alex'}

In [305]:
p.first_name

calling __get__ for first_name


'Alex'

In [306]:
class IntegerValue:
    def __set__(self, instance, value):
        print('__set__ called...')
        
    def __get__(self, instance, owner_class):
        print('__get__ called...')

In [307]:
class Point:
    x = IntegerValue()

In [308]:
p = Point()

In [309]:
p.x = 100

__set__ called...


In [310]:
p.x

__get__ called...


In [311]:
p.__dict__

{}

In [312]:
p.__dict__['x'] = 'hello'

In [313]:
p.__dict__

{'x': 'hello'}

In [314]:
p.x

__get__ called...


In [315]:
p.x = 100

__set__ called...


In [316]:
from datetime import datetime

class TimeUTC:
    def __get__(self, instance, owner_class):
        print('__get__ called...')
        return datetime.utcnow().isoformat()

In [317]:
class Logger:
    current_time = TimeUTC()

In [318]:
l = Logger()

In [319]:
l.current_time

__get__ called...


'2024-12-11T02:33:25.519874'

In [320]:
l.__dict__

{}

In [321]:
l.__dict__['current_time'] = 'this is not a timestamp'

In [322]:
l.__dict__

{'current_time': 'this is not a timestamp'}

In [323]:
l.current_time

'this is not a timestamp'

In [324]:
class ValidString:
    def __init__(self, min_length):
        self.min_length = min_length
        
    def __set_name__(self, owner_class, prop_name):
        self.prop_name = prop_name
        
    def __set__(self, instance, value):
        if not isinstance(value, str):
            raise ValueError(f'{self.prop_name} must be a string.')
        if len(value) < self.min_length:
            raise ValueError(f'{self.prop_name} must be '
                             f'at least {self.min_length} characters.'
                            )
        instance.__dict__[self.prop_name] = value
        
    def __get__(self, instance, owner_class):
        if instance is None:
            return self
        else:
            return instance.__dict__.get(self.prop_name, None)

In [325]:
class Person:
    first_name = ValidString(1)
    last_name = ValidString(2)

In [326]:
p = Person()

In [327]:
p.__dict__

{}

In [328]:
p.first_name = 'Alex'
p.last_name = 'Martelli'

In [329]:
p.__dict__

{'first_name': 'Alex', 'last_name': 'Martelli'}

In [330]:
p.first_name, p.last_name

('Alex', 'Martelli')

# Properties & Descriptors

In [331]:
from numbers import Integral

class Person:
    @property
    def age(self):
        return getattr(self, '_age', None)
    
    @age.setter
    def age(self, value):
        if not isinstance(value, Integral):
            raise ValueError('age: must be an integer.')
        if value < 0:
            raise ValueError('age: must be a non-negative integer.')
        self._age = value

In [332]:
p = Person()

In [333]:
try:
    p.age = -10
except ValueError as ex:
    print(ex)

age: must be a non-negative integer.


In [334]:
p.age = 10

In [335]:
p.age, p.__dict__

(10, {'_age': 10})

In [336]:
class Person:
    def get_age(self):
        return getattr(self, '_age', None)
    
    def set_age(self, value):
        if not isinstance(value, Integral):
            raise ValueError('age: must be an integer.')
        if value < 0:
            raise ValueError('age: must be a non-negative integer.')
        self._age = value
        
    age = property(fget=get_age, fset=set_age)

In [337]:
p = Person()

In [338]:
try:
    p.age = -10
except ValueError as ex:
    print(ex)

age: must be a non-negative integer.


In [339]:
p.age = 10

In [340]:
p.age, p.__dict__

(10, {'_age': 10})

In [341]:
prop = Person.age

In [342]:
prop

In [343]:
hasattr(prop, '__set__')

True

In [344]:
hasattr(prop, '__get__')

True

In [345]:
from datetime import datetime

class TimeUTC:
    @property
    def current_time(self):
        return datetime.utcnow().isoformat()

In [346]:
t = TimeUTC()
t.current_time

'2024-12-11T02:40:06.169337'

In [347]:
prop = TimeUTC.current_time

In [348]:
hasattr(prop, '__get__')

True

In [349]:
hasattr(prop, '__set__')

True

In [350]:
try:
    t.current_time = datetime.utcnow().isoformat()
except AttributeError as ex:
    print(ex)

can't set attribute


In [351]:
t.__dict__

{}

In [352]:
t.__dict__['current_time'] = 'not a time'

In [353]:
t.__dict__

{'current_time': 'not a time'}

In [354]:
t.current_time

'2024-12-11T02:40:52.316009'

In [355]:
class MakeProperty:
    def __init__(self, fget=None, fset=None):
        self.fget = fget
        self.fset = fset
        
    def __set_name__(self, owner_class, prop_name):
        self.prop_name = prop_name
        
    def __get__(self, instance, owner_class):
        print('__get__ called...')
        if instance is None:
            return self
        if self.fget is None:
            raise AttributeError(f'{self.prop_name} is not readable.')
        return self.fget(instance)
            
    def __set__(self, instance, value):
        print('__set__ called...')
        if self.fset is None:
            raise AttributeError(f'{self.prop_name} is not writable.')
        self.fset(instance, value)

In [356]:
class Person:
    def get_name(self):
        return self._name
    
    def set_name(self, value):
        self._name = value
        
    name = MakeProperty(fget=get_name, fset=set_name)

In [357]:
p = Person()

In [358]:
p.__dict__

{}

In [359]:
p.name = 'Guido'

__set__ called...


In [360]:
p.name

__get__ called...


'Guido'

In [361]:
p.__dict__['name'] = 'Alex'

In [362]:
p.__dict__

{'_name': 'Guido', 'name': 'Alex'}

In [363]:
p.name

__get__ called...


'Guido'

In [364]:
class MakeProperty:
    def __init__(self, fget=None, fset=None):
        self.fget = fget
        self.fset = fset
        
    def __set_name__(self, owner_class, prop_name):
        self.prop_name = prop_name
        
    def __get__(self, instance, owner_class):
        print('__get__ called...')
        if instance is None:
            return self
        if self.fget is None:
            raise AttributeError(f'{self.prop_name} is not readable.')
        return self.fget(instance)
            
    def __set__(self, instance, value):
        print('__set__ called...')
        if self.fset is None:
            raise AttributeError(f'{self.prop_name} is not writable.')
        self.fset(instance, value)
        
    def setter(self, fset):
        self.fset = fset
        return self
        

In [365]:
class Person:
    def get_first_name(self):
        return getattr(self, '_first_name', None)
    
    def set_first_name(self, value):
        self._first_name = value
        
    def get_last_name(self):
        return getattr(self, '_last_name', None)
    
    def set_last_name(self, value):
        self._last_name = value
        
    first_name = MakeProperty(fget=get_first_name, fset=set_first_name)
    last_name = MakeProperty(fget=get_last_name, fset=set_last_name)

In [366]:
class Person:
    @MakeProperty
    def first_name(self):
        return getattr(self, '_first_name', None)
    
    @first_name.setter
    def first_name(self, value):
        self._first_name = value
        
    @MakeProperty
    def last_name(self):
        return getattr(self, '_last_name', None)
    
    @last_name.setter
    def last_name(self, value):
        self._last_name = value

In [367]:
p1 = Person()

In [368]:
p1.first_name = 'Raymond'

__set__ called...


In [369]:
p1.last_name = 'Hettinger'

__set__ called...


In [370]:
p1.first_name

__get__ called...


'Raymond'

In [371]:
p1.last_name

__get__ called...


'Hettinger'

In [372]:
p2 = Person()
p2.first_name, p2.last_name = 'Alex', 'Martelli'

__set__ called...
__set__ called...


In [373]:
p1.first_name, p1.last_name, p2.first_name, p2.last_name

__get__ called...
__get__ called...
__get__ called...
__get__ called...


('Raymond', 'Hettinger', 'Alex', 'Martelli')

# Examples

In [374]:
class Int:
    def __set_name__(self, owner_class, prop_name):
        self.prop_name = prop_name
        
    def __set__(self, instance, value):
        if not isinstance(value, int):
            raise ValueError(f'{self.prop_name} must be an integer.')
        instance.__dict__[self.prop_name] = value
        
    def __get__(self, instance, owner_class):
        if instance is None:
            return self
        else:
            return instance.__dict__.get(self.prop_name, None)
            

In [375]:
class Float:
    def __set_name__(self, owner_class, prop_name):
        self.prop_name = prop_name
        
    def __set__(self, instance, value):
        if not isinstance(value, float):
            raise ValueError(f'{self.prop_name} must be a float.')
        instance.__dict__[self.prop_name] = value
        
    def __get__(self, instance, value):
        if instance is None:
            return self
        else:
            return instance.__dict__.get(self.prop_name, None)

In [376]:
class List:
    def __set_name__(self, owner_class, prop_name):
        self.prop_name = prop_name
        
    def __set__(self, instance, value):
        if not isinstance(value, list):
            raise ValueError(f'{self.prop_name} must be a list.')
        instance.__dict__[self.prop_name] = value
        
    def __get__(self, instance, value):
        if instance is None:
            return self
        else:
            return instance.__dict__.get(self.prop_name, None)
        
    

In [377]:
class Person:
    age = Int()
    height = Float()
    tags = List()
    favorite_foods = List()

In [378]:
p = Person()

In [379]:
try:
    p.age = 12.5
except ValueError as ex:
    print(ex)

age must be an integer.


In [380]:
try:
    p.height = 'abc'
except ValueError as ex:
    print(ex)

height must be a float.


In [381]:
try:
    p.tags = 'python'
except ValueError as ex:
    print(ex)

tags must be a list.


In [382]:
class ValidType:
    def __init__(self, type_):
        self._type = type_
        
    def __set_name__(self, owner_clasds, prop_name):
        self.prop_name = prop_name
        
    def __set__(self, instance, value):
        if not isinstance(value, self._type):
            raise ValueError(f'{self.prop_name} must be of type '
                             f'{self._type.__name__}'
                            )
        instance.__dict__[self.prop_name] = value
        
    def __get__(self, instance, owner_class):
        if instance is None:
            return self
        else:
            return instance.__dict__.get(self.prop_name, None)

In [383]:
class Person:
    age = ValidType(int)
    height = ValidType(float)
    tags = ValidType(list)
    favorite_foods = ValidType(tuple)
    name = ValidType(str)

In [384]:
p = Person()

In [385]:
try:
    p.age = 10.5
except ValueError as ex:
    print(ex)

age must be of type int


# Example 2

In [386]:
class Int:
    def __init__(self, min_value=None, max_value=None):
        self.min_value = min_value
        self.max_value = max_value
        
    def __set_name__(self, owner_class, name):
        self.name = name
        
    def __set__(self, instance, value):
        if not isinstance(value, int):
            raise ValueError(f'{self.name} must be an int.')
        if self.min_value is not None and value < self.min_value:
            raise ValueError(f'{self.name} must be at least {self.min_value}')
        if self.max_value is not None and value > self.max_value:
            raise ValueError(f'{self.name} cannot exceed {self.max_value}')
        instance.__dict__[self.name] = value
        
    def __get__(self, instance, owner_class):
        if instance is None:
            return self
        else:
            return instance.__dict__.get(self.name, None)

In [387]:
class Point2D:
    x = Int(min_value=0, max_value=800)
    y = Int(min_value=0, max_value=400)
    
    def __init__(self, x, y):
        self.x = x
        self.y = y
        
    def __repr__(self):
        return f'Point2D(x={self.x}, y={self.y})'
    
    def __str__(self):
        return f'({self.x}, {self.y})'

In [388]:
p = Point2D(0, 10)

In [389]:
str(p)

'(0, 10)'

In [390]:
repr(p)

'Point2D(x=0, y=10)'

In [391]:
p.x, p.y

(0, 10)

In [392]:
try:
    p = Point2D(0, 500)
except ValueError as ex:
    print(ex)

y cannot exceed 400


In [393]:
import collections

In [394]:
isinstance([1, 2, 3], collections.abc.Sequence)

True

In [395]:
isinstance([1, 2, 3], collections.abc.MutableSequence)

True

In [396]:
isinstance((1, 2, 3), collections.abc.Sequence)

True

In [397]:
isinstance((1, 2, 3), collections.abc.MutableSequence)

False

In [398]:
class Point2DSequence:
    def __init__(self, min_length=None, max_length=None):
        self.min_length = min_length
        self.max_length = max_length
        
    def __set_name__(self, cls, name):
        self.name = name
        
    def __set__(self, instance, value):
        if not isinstance(value, collections.abc.Sequence):
            raise ValueError(f'{self.name} must be a sequence type.')
        if self.min_length is not None and len(value) < self.min_length:
            raise ValueError(f'{self.name} must contain at least '
                             f'{self.min_length} elements'
                            )
        if self.max_length is not None and len(value) > self.max_length:
            raise ValueError(f'{self.name} cannot contain more than  '
                             f'{self.max_length} elements'
                            )
        for index, item in enumerate(value):
            if not isinstance(item, Point2D):
                raise ValueError(f'Item at index {index} is not a Point2D instance.')
                
        # value passes checks - want to store it as a mutable sequence so we can 
        # append to it later
        instance.__dict__[self.name] = list(value)
        
    def __get__(self, instance, cls):
        if instance is None:
            return self
        else:
            if self.name not in instance.__dict__:
                # current point list has not been defined,
                # so let's create an empty list
                instance.__dict__[self.name] = []
            return instance.__dict__.get(self.name)

In [399]:
class Polygon:
    vertices = Point2DSequence(min_length=3)
    
    def __init__(self, *vertices):
        self.vertices = vertices

In [400]:
try:
    p = Polygon()
except ValueError as ex:
    print(ex)

vertices must contain at least 3 elements


In [401]:
try:
    p = Polygon(Point2D(-100,0), Point2D(0, 1), Point2D(1, 0))
except ValueError as ex:
    print(ex)

x must be at least 0


In [402]:
p = Polygon(Point2D(0,0), Point2D(0, 1), Point2D(1, 0))

In [403]:
p.vertices

[Point2D(x=0, y=0), Point2D(x=0, y=1), Point2D(x=1, y=0)]

In [404]:
class Polygon:
    vertices = Point2DSequence(min_length=3)
    
    def __init__(self, *vertices):
        self.vertices = vertices
        
    def append(self, pt):
        if not isinstance(pt, Point2D):
            raise ValueError('Can only append Point2D instances.')
        max_length = type(self).vertices.max_length
        if max_length is not None and len(self.vertices) >= max_length:
            # cannot add more points!
            raise ValueError(f'Vertices length is at max ({max_length})')
        self.vertices.append(pt)
                

In [405]:
p = Polygon(Point2D(0,0), Point2D(1,0), Point2D(0,1))

In [406]:
p.vertices

[Point2D(x=0, y=0), Point2D(x=1, y=0), Point2D(x=0, y=1)]

In [407]:
p.append(Point2D(10, 10))

In [408]:
p.vertices

[Point2D(x=0, y=0), Point2D(x=1, y=0), Point2D(x=0, y=1), Point2D(x=10, y=10)]

In [409]:
class Polygon:
    vertices = Point2DSequence(min_length=3, max_length=3)
    
    def __init__(self, *vertices):
        self.vertices = vertices
        
    def append(self, pt):
        if not isinstance(pt, Point2D):
            raise ValueError('Can only append Point2D instances.')
        max_length = type(self).vertices.max_length
        if max_length is not None and len(self.vertices) >= max_length:
            # cannot add more points!
            raise ValueError(f'Vertices length is at max ({max_length})')
        self.vertices.append(pt)
                

In [410]:
p = Polygon(Point2D(0,0), Point2D(1,0), Point2D(0,1))

In [411]:
try:
    p.append(Point2D(10, 10))
except ValueError as ex:
    print(ex)

Vertices length is at max (3)


# Functions and Descriptors

In [412]:
def add(a, b):
    return a + b

In [413]:
hasattr(add, '__get__')

True

In [414]:
import sys

In [415]:
me = sys.modules['__main__']

In [416]:
p = add.__get__(None, me)

In [417]:
p, id(p)

(<function __main__.add(a, b)>, 2229758195744)

In [418]:
add, id(add)

(<function __main__.add(a, b)>, 2229758195744)

In [419]:
class Person:
    def __init__(self, name):
        self.name = name
        
    def say_hello(self):
        return f'{self.name} says hello'

In [420]:
Person.say_hello

<function __main__.Person.say_hello(self)>

In [421]:
p = Person('Alex')

In [422]:
hex(id(p))

'0x20727baf130'

In [423]:
p.say_hello

<bound method Person.say_hello of <__main__.Person object at 0x0000020727BAF130>>

In [424]:
bound_method = Person.say_hello.__get__(p, Person)

In [425]:
bound_method

<bound method Person.say_hello of <__main__.Person object at 0x0000020727BAF130>>